In [1]:
import sys
import os
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import rcParams, cycler
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import xarray as xr
import geopandas as gpd
import rioxarray
from xeofs.xarray import EOF
import gc
import collections
import statsmodels.api as sm
import scipy
from sklearn.metrics import mean_squared_error
from math import sqrt
from itertools import product
import pyproj
from shapely.geometry import mapping
import pandas as pd

In [2]:
# Define project repo path
inDirName = '/Users/smurugan9/research/aislens/aislens_emulation/'

# DATA FILE PATHS

# Data containing regridded flux and SSH for 150 years
regriddedFluxSSH_filepath = 'data/interim/RegriddedFluxSSH.nc'

# File contains all defined ice shelves
iceShelvesShape_filepath = 'data/interim/iceShelves.geojson'

# Folder for output figures
figures_folderpath = 'reports/figures/' 

interim_data_folder = 'data/interim/'
flux_dedrafted_data_path = 'dedrafted_flux_IMBIE/'
randomized_realizations_path = 'randomized_realizations/'
flux_dedrafted_iceshelves_data_path = 'iceshelves_dedrafted_flux/'
reconstructions_neofs_path = 'reconstructions_neofs/'

In [3]:
# Read geoJSON region feature file as GeoDataFrame
iceshelvesmask = gpd.read_file(inDirName + iceShelvesShape_filepath)
# Convert to south polar stereographic projection
icems = iceshelvesmask.to_crs({'init': 'epsg:3031'});
crs = ccrs.SouthPolarStereo();

/Users/smurugan9/opt/anaconda3/envs/aislens/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [4]:
# Input data files

regridded_data = xr.open_dataset(inDirName+regriddedFluxSSH_filepath)
flux = regridded_data.timeMonthly_avg_landIceFreshwaterFlux
ssh = regridded_data.timeMonthly_avg_ssh

flux_clean = xr.open_dataset(inDirName+interim_data_folder+'flux_clean')
flux_clean = flux_clean.timeMonthly_avg_landIceFreshwaterFlux


eofs = xr.open_dataset(inDirName+interim_data_folder+"fluxclean_eofs.nc")
pcs = xr.open_dataset(inDirName+interim_data_folder+"fluxclean_pcs.nc")
varexpl = xr.open_dataset(inDirName+interim_data_folder+"fluxclean_varexpl.nc")

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(icems.name)

0           Filchner-Ronne
1                     Ross
2               Antarctica
3                Peninsula
4          West Antarctica
5          East Antarctica
6                   IMBIE1
7                   IMBIE2
8                   IMBIE3
9                   IMBIE4
10                  IMBIE5
11                  IMBIE6
12                  IMBIE7
13                  IMBIE8
14                  IMBIE9
15                 IMBIE10
16                 IMBIE11
17                 IMBIE12
18                 IMBIE13
19                 IMBIE14
20                 IMBIE15
21                 IMBIE16
22                 IMBIE17
23                 IMBIE18
24                 IMBIE19
25                 IMBIE20
26                 IMBIE21
27                 IMBIE22
28                 IMBIE23
29                 IMBIE24
30                 IMBIE25
31                 IMBIE26
32                 IMBIE27
33                   Abbot
34                   Amery
35                    Atka
36                 Aviator
3

In [ ]:
snscolors = sns.color_palette("deep")

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


fig, ax = plt.subplots(1, 1, figsize=[15, 15],subplot_kw={'projection': ccrs.SouthPolarStereo()})

# Create inset of width 1.3 inches and height 0.9 inches
# at the default upper right location
axins = inset_axes(ax, width="20%", height="20%")

# Create inset of width 30% and height 40% of the parent axes' bounding box
# at the lower left corner (loc=3)
axins2 = inset_axes(ax, width="20%", height="20%", loc=2)

# Create inset of mixed specifications in the second subplot;
# width is 30% of parent axes' bounding box and
# height is 1 inch at the upper left corner (loc=2)
axins3 = inset_axes(ax, width="20%", height="20%", loc=3, borderpad=0)

# Create an inset in the lower right corner (loc=4) with borderpad=1, i.e.
# 10 points padding (as 10pt is the default fontsize) to the parent axes
axins4 = inset_axes(ax, width="20%", height="20%", loc=4)

# Turn ticklabels of insets off
#for axi in [axins, axins2, axins3, axins4]:
#    axi.tick_params(labelleft=False, labelbottom=False)

    
#flux_clean[1].plot(ax=ax)

#plt.figure(figsize=(10,10))
#ax = plt.subplot(111,projection=ccrs.SouthPolarStereo())
#ax.gridlines()
icems[34:133].boundary.plot(ax=ax,linewidth=0.5,color=snscolors[5])
ax.coastlines(resolution='10m', zorder=6,linewidth=0.75)
icems.loc[[basins[0]],'geometry'].plot(ax=ax,color=snscolors[0])
icems.loc[[basins[1]],'geometry'].plot(ax=ax,color=snscolors[1])
icems.loc[[basins[2]],'geometry'].plot(ax=ax,color=snscolors[2])
icems.loc[[basins[3]],'geometry'].plot(ax=ax,color=snscolors[3])
icems.loc[[basins[4]],'geometry'].plot(ax=ax,color=snscolors[4])

siz=2

axins.scatter(basins_fl_mean[0],ssh[1],color=snscolors[0],s=siz)
axins.set_title(icems.name[basins[0]])

axins2.scatter(basins_fl_mean[1],ssh[1],color=snscolors[1],s=siz)
axins2.set_title(icems.name[basins[1]])

axins3.scatter(basins_fl_mean[2],ssh[1],color=snscolors[2],s=siz)
axins3.set_title(icems.name[basins[2]])

axins4.scatter(basins_fl_mean[3],ssh[1],color=snscolors[3],s=siz)
axins4.set_title(icems.name[basins[3]])

plt.show()

In [ ]:
basins = np.array((34,55,114,103,1))

In [6]:
for basin in basins:
    print(icems.name[basin])

NameError: name 'basins' is not defined

In [38]:
basins = np.array((34,55,114,103,1))
basins_fl_mean = np.empty((basins.shape[0],flux_clean.shape[1],flux_clean.shape[2]))
i=0
for basin in basins:
    basin_flux = flux_clean.rio.clip(icems.loc[[basin],'geometry'].apply(mapping),icems.crs,drop=False)
    basins_fl_mean[i] = basin_flux.mean('time')
    i=i+1
    del basin_flux
    gc.collect()

In [8]:
#basinsall = icems.shape[0]
basins_fl_mean_all = np.empty((icems.shape[0],flux_clean.shape[1],flux_clean.shape[2]))
i=0
for basin in range(33,133):
    print('selecting basin for {}'.format(basin))
    basin_flux = flux_clean.rio.clip(icems.loc[[basin],'geometry'].apply(mapping),icems.crs,drop=False)
    print('clipped data for catchment {}'.format(basin))
    basins_fl_mean_all[i] = basin_flux.mean('time')
    i=i+1
    del basin_flux
    gc.collect()

np.save(inDirName+interim_data_folder+"basins_melt_scatter.npy",basins_fl_mean_all)

selecting basin for 33
clipped data for catchment 33
selecting basin for 34
clipped data for catchment 34
selecting basin for 35
clipped data for catchment 35
selecting basin for 36
clipped data for catchment 36
selecting basin for 37
clipped data for catchment 37
selecting basin for 38
clipped data for catchment 38
selecting basin for 39
clipped data for catchment 39
selecting basin for 40
clipped data for catchment 40
selecting basin for 41
clipped data for catchment 41
selecting basin for 42
clipped data for catchment 42
selecting basin for 43
clipped data for catchment 43
selecting basin for 44
clipped data for catchment 44
selecting basin for 45
clipped data for catchment 45
selecting basin for 46
clipped data for catchment 46
selecting basin for 47
clipped data for catchment 47
selecting basin for 48
clipped data for catchment 48
selecting basin for 49
clipped data for catchment 49
selecting basin for 50
clipped data for catchment 50
selecting basin for 51
clipped data for catchm

In [28]:
np.save(inDirName+interim_data_folder+"catchments_scatter.npy",catchments)

In [ ]:
basin_flux_tmean = basin_flux.mean('time')
plt.scatter(basin_flux_tmean,ssh[1])

In [22]:
catchments = basins_fl_mean_all[0:100]